In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df  = pd.read_csv("C://Users//Jibin Im//Downloads//creditcard.csv")[:80_000]

df.head()



,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [2]:
X = df.drop(columns=['Time', 'Amount', 'Class']).values
y = df['Class'].values
y.sum()

196

In [3]:
from sklearn.linear_model import LogisticRegression

mod = LogisticRegression(class_weight={0:1, 1:2}, max_iter=1000)#class_weight={0:1, 1:2} puts more weifght if a fraud is found. if not fraud, its 1 and if it is fraud a weight of 2
mod.fit(X, y).predict(X).sum()                  


172

In [4]:
from sklearn.ensemble import IsolationForest
from collections import Counter

mod = IsolationForest().fit(X)
#Counter(mod.predict(X)) # this tells me how many are outliers. -1 being an outlier and 1 being not an outlier
np.where(mod.predict(X)== -1, 1, 0)#where mod.predict returns a -1, i want it to be equal to 1 to signal a positive outlier, otherwise, 0

array([0, 0, 0, ..., 0, 0, 0])

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, make_scorer

def min_recall_precision(est, y_true, X, sample_weight = None):
    y_pred = est.predict(X)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    return min(recall, precision)

def outlier_precision(mod, X, y):
    pred = mod.predict(X)
    return precision_score(y, np.where(pred)==-1, 1, 0)

def outlier_recall(mod, X, y):
    pred = mod.predict(X)
    return recall_score(y, np.where(pred)==-1, 1, 0)

#Use precision when
#When the cost of false positives is high. For example, in spam detection, sending legitimate emails to the spam folder is costly.
#When you want to ensure that most of the predicted positive instances are correct.

#Use Recall When
#When the cost of false negatives is high. For example, in medical diagnostics, missing a disease case (false negative) can be very costly.
#When it is crucial to identify all positive instances, even at the risk of some false positives.
grid = GridSearchCV(
    estimator=LogisticRegression(max_iter=1000),
    param_grid={'class_weight':[{0:1, 1:v} for v in np.linspace(1, 20, 30)]},
    scoring = {'precision': outlier_precision,'recall_score': outlier_recall},

    refit='precision',
    cv = 10,
    n_jobs=-1
)
grid.fit(X, y)
pd.DataFrame(grid.cv_results_)

c:\Users\Jibin Im\anaconda3\envs\pytorch\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,params,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,...,split3_test_recall_score,split4_test_recall_score,split5_test_recall_score,split6_test_recall_score,split7_test_recall_score,split8_test_recall_score,split9_test_recall_score,mean_test_recall_score,std_test_recall_score,rank_test_recall_score
0,0.552838,0.305004,0.229170,0.232579,"{0: 1, 1: 1.0}","{'class_weight': {0: 1, 1: 1.0}}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,0.558343,0.288539,0.149057,0.276090,"{0: 1, 1: 1.6551724137931034}","{'class_weight': {0: 1, 1: 1.6551724137931034}}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,0.538513,0.239030,0.005922,0.002090,"{0: 1, 1: 2.310344827586207}","{'class_weight': {0: 1, 1: 2.310344827586207}}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,0.522856,0.218953,0.005923,0.000895,"{0: 1, 1: 2.9655172413793105}","{'class_weight': {0: 1, 1: 2.9655172413793105}}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,0.732801,0.504873,0.006825,0.002303,"{0: 1, 1: 3.6206896551724137}","{'class_weight': {0: 1, 1: 3.6206896551724137}}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,0.745759,0.555016,0.013295,0.010906,"{0: 1, 1: 4.275862068965517}","{'class_weight': {0: 1, 1: 4.275862068965517}}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,0.857081,0.556764,0.007381,0.003308,"{0: 1, 1: 4.931034482758621}","{'class_weight': {0: 1, 1: 4.931034482758621}}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,0.682936,0.418328,0.006420,0.002241,"{0: 1, 1: 5.586206896551724}","{'class_weight': {0: 1, 1: 5.586206896551724}}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,0.902182,0.455521,0.005862,0.001862,"{0: 1, 1: 6.241379310344827}","{'class_weight': {0: 1, 1: 6.241379310344827}}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,0.742732,0.431626,0.006374,0.002093,"{0: 1, 1: 6.896551724137931}","{'class_weight': {0: 1, 1: 6.896551724137931}}",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
